In [1]:
import pandas as pd
import sqlite3

connection = sqlite3.connect("base_de_datos_ferreteria.db")
cursor = connection.cursor()

def df_sql(query):
    return pd.read_sql(query, connection)

## Consulta de Saldo por Mes

In [2]:
def consulta_saldo(mes=None):
    query = '''
    SELECT STRFTIME ("%m", A.Fecha) Mes,
        SUM(A.Precio) Total_Ventas,
        SUM(C.Precio) Total_Compras,
        (SUM(A.Precio) - SUM(C.Precio)) Saldo
    FROM ventas A
    INNER JOIN producto B ON A.Producto = B.Id_producto
    INNER JOIN compras C ON C.Producto = B.Id_producto
    GROUP BY Mes
    '''
    saldo = df_sql(query)
    return saldo[saldo['Mes'] == str(mes).rjust(2, '0')] if mes else saldo

consulta_saldo(10)

,Mes,Total_Ventas,Total_Compras,Saldo
9,10,195639.25,142767.02,52872.23


## Consulta de Stock por Producto

In [3]:
def consulta_stock(producto=None):
    query = '''
    SELECT p.Nombre, cat.Nombre AS categoria, (SUM(c.Cantidad) - SUM(v.Cantidad)) AS stock
    FROM producto P
    LEFT JOIN categoria cat ON p.Categoria = cat.Id_categoria
    LEFT JOIN compras c ON p.Id_producto = c.Producto
    LEFT JOIN ventas v ON p.Id_producto = v.Producto
    GROUP BY p.Nombre, cat.Nombre;
    '''
    stock = df_sql(query)
    return stock[stock['Nombre'] == producto] if producto else stock

consulta_stock('Alicate')

,Nombre,categoria,stock
0,Alicate,Herramientas manuales,-60


In [6]:
connection.close()